In [3]:
#Установка нужных пакетов
#!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Stas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Stas\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# выгружаем датасет:
#!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[0]))

['Can', 'I', 'get', 'back', 'with', 'my', 'ex', 'even', 'though', 'she', 'is', 'pregnant', 'with', 'another', 'guy', "'", 's', 'baby', '?']


In [6]:
#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok

#data[i]

data_tok = []
for i in range(len(data)):
  data_tok.append(tokenizer.tokenize(data[i].lower()))

#checking
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [7]:
###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data_tok

dict_words = {}

for token in data_tok:
  for word in token:
    if word in dict_words.keys():
      dict_words[word] += 1
    else:
      dict_words[word] = 1

import pandas as pd
df = pd.DataFrame.from_dict(dict_words, orient='index')
df
df.sort_values(0, ascending = False).head(10)

,0
?,552413
the,252068
what,214798
is,185392
a,155726
i,149740
to,141788
in,139787
how,135687
of,112001


REGEXP

https://www.programiz.com/python-programming/regex

https://docs.python.org/3/howto/regex.html

In [ ]:
import re

pattern = 'a.*s'
test_string = 'abyss'
result = re.match(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")

Bag Of Words (BOW)

In [8]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input
    and returns a vector of size of filtered_vocab.It puts 0 if the
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector

def unique(sequence):
    '''This functions returns a list in which the order remains
    same and no item repeats.Using the set() function does not
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

#Write the sentences in the corpus,in our case, just two
string1="Welcome to Great Learning , Now start learning"
string2="Learning is a good practice"

#convert them to lower case
string1=string1.lower()
string2=string2.lower()

#split the sentences into tokens
tokens1=string1.split()
tokens2=string2.split()
print(tokens1)
print(tokens2)

#create a vocabulary list
vocab=unique(tokens1+tokens2)
print(vocab)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab:
    if w not in stopwords and w not in special_char:
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
vector1=vectorize(tokens1)
print(vector1)
vector2=vectorize(tokens2)
print(vector2)

['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'learning']
['learning', 'is', 'a', 'good', 'practice']
['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'is', 'a', 'good', 'practice']
['welcome', 'great', 'learning', 'now', 'start', 'good', 'practice']
[1, 1, 2, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]


Задание 10: Реализовать Bag of words на data_tok (можно на NLTK, можно без)

In [11]:
def filtered(data):
    return [w for w in data if w not in stopwords and w not in special_char]


data = data_tok[:100]
vocab = unique([ps.stem(word) for line in data for word in line])
print('vocab:', len(vocab))
filtered_vocab = filtered(vocab)
print('filtered:', len(filtered_vocab))
vectors = [vectorize(line) for line in data]

vocab: 513
filtered: 506


In [13]:
from numpy import dot
from numpy.linalg import norm


a = vectors[0]
b = vectors[10]
cos_sim = dot(a, b)/(norm(a)*norm(b))

from numpy import dot
from numpy.linalg import norm

result = []
for i in range(len(vectors)):
    for j in range(len(vectors)):
        if vectors[i] != vectors[j]:
            cos_sim = dot (vectors[i], vectors[j])/( norm (vectors[i]) * norm (vectors[j]))
            result += [[cos_sim, data[i], data[j]]]

sorted(result, key=lambda x: x[0], reverse=True)[0]



[0.8944271909999159,
 ['what', 'are', 'the', 'application', 'of', 'binary', 'search', 'trees', '?'],
 ['what', 'are', 'the', 'components', 'of', 'fitness', '?']]

In [14]:
from numpy import dot
from numpy.linalg import norm

a = vectors[0]
b = vectors[2]
cos_sim = dot(a, b)/(norm(a)*norm(b))

cos_sim

0.0

Готовые модели

In [ ]:
import gensim.downloader as api
model = api.load('glove-twitter-25')
model.most_similar(positive=["coder", "money"])

# , negative=["brain"]

# model.most_similar(positive=["spain","moscow"], negative=["quenn"])

In [ ]:
words = sorted(model.vocab.keys(),
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])